In [11]:
import os
import asyncio
import threading
import time
import base64
from datetime import datetime, timedelta
from datetime import time
from typing import List, Dict, Any, Optional
import sqlite3
from contextlib import asynccontextmanager
import json
from fastapi.responses import FileResponse
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage
import base64


from fastapi import FastAPI, HTTPException, BackgroundTasks, Depends, File, UploadFile, Form
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel, Field, validator
from dotenv import load_dotenv
import uvicorn
#from playsound import playsound
import io

# LangGraph and LangChain imports
from langgraph.graph import StateGraph, END, START
from langchain_core.messages import HumanMessage
from langchain_groq import ChatGroq
from fastapi.staticfiles import StaticFiles 
from typing import TypedDict,Annotated,Literal
from langchain.prompts import PromptTemplate

from twilio.rest import Client
from twilio.base.exceptions import TwilioException
from datetime import time as dtime
from fastapi import FastAPI, File, UploadFile, Form, HTTPException
from fastapi.responses import JSONResponse
from pydantic import BaseModel, Field
from typing import Optional, Literal
import base64
from datetime import datetime
import uvicorn
from pydantic import field_validator
from typing_extensions import Literal

#import nest_asyncio
#nest_asyncio.apply()
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

True

We are going to Define States

In [12]:
class Location(BaseModel):
    latitude:  float = Field(..., ge=-90.0, le=90.0, description="Degrees north or south of the equator")
    longitude: float = Field(..., ge=-180.0, le=180.0, description="Degrees east or west of prime meridian")

    # Optional: custom string pattern instead of float
    @field_validator("latitude", "longitude")
    def parse_str_to_float(cls, v):
        # Allow passing "12.345" or 12.345
        return float(v)


In [13]:
class Summarization(BaseModel):
    summary: str
    location: Location
    time: datetime
    distress_words: List[str]
    threat_level: float


In [14]:
class WorkflowState(BaseModel):
    location:    Location
    real_time:   datetime
    TAudio:      Optional[str]=None
    threat_score:float=0.0
    consent:     Literal["Yes", "No"]
    gesture:     Literal["Yes", "No"]
    sudden_acc:  Literal["Yes", "No"]
    message:str
    analysis: Optional[Summarization] = None
    audio_b64:str|None


Just an Example

Making an Sqlit3 Database and attaching it

In [15]:
import sqlite3
#Database_URL="Log_of_Events.db"
class DatabaseManager:
    def __init__(self,db_path:str):
        self.db_path=db_path
        self.init_database()


    def init_database(self):
        with sqlite3.connect(self.db_path) as conn:
            conn.execute("""
CREATE TABLE IF NOT EXISTS Inputs_Recieved(
    real_time TEXT PRIMARY KEY,
    location TEXT NOT NULL,
    TAudio TEXT,
    consent TEXT NOT NULL CHECK (consent IN('Yes','No')),
    gesture TEXT NOT NULL CHECK (gesture IN('Yes','No')),
    sudden_acc TEXT NOT NULL CHECK(sudden_acc IN('Yes','No')),
    message TEXT)
                         """)
            conn.execute("""
CREATE TABLE IF NOT EXISTS SUMMARY_REPORT(
    real_time TEXT PRIMARY KEY,
    threat_level REAL NOT NULL,
    summary TEXT NOT NULL,
    distress_words TEXT NOT NULL,                                                          
    location TEXT NOT NULL,
    audio TEXT)
""")
            conn.commit()
    def store_summary(self,real_time:datetime,threat_level:float,summary:str,distress_words:list[str],location:Location,audio:str=None):
        import json
        loc_json=json.dumps(location.dict())
        with sqlite3.connect(self.db_path) as conn:
            distress_json = json.dumps(distress_words)
            real_time_str = real_time.isoformat() if isinstance(real_time, datetime) else str(real_time)  # fix for Change 6


            if audio is None:
               conn.execute("""
               INSERT INTO SUMMARY_REPORT
               (real_time,threat_level,summary,distress_words,location)
                            VALUES(?,?,?,?,?)             
                            """,(real_time_str,threat_level,summary,distress_json,loc_json))

            else:
                conn.execute("""
               INSERT INTO SUMMARY_REPORT
               (real_time,threat_level,summary,distress_words,location,audio)
                            VALUES(?,?,?,?,?,?)             
                            """,(real_time_str,threat_level,summary,distress_json,loc_json,audio))
            conn.commit()
    def store_logs(
        self,
        real_time: datetime,
        location,           # Location object
        TAudio: Optional[str],
        consent: str,
        gesture: str,
        sudden_acc: str,
        message: str
    ):
        # Convert Location to JSON string
        location_json = json.dumps(location.dict())

        with sqlite3.connect(self.db_path) as conn:
            conn.execute("""
                INSERT INTO Inputs_Recieved 
                (real_time, location, TAudio, consent, gesture, sudden_acc, message)
                VALUES (?, ?, ?, ?, ?, ?, ?)
            """, (
                real_time.isoformat(),
                location_json,
                TAudio,
                consent,
                gesture,
                sudden_acc,
                message
            ))
            conn.commit() 

    def get_summarization(self,real_time:datetime)->Optional[dict[str,Any]]:
        with sqlite3.connect(self.db_path) as conn:
            conn.row_factory=sqlite3.Row
            cursor=conn.cursor()
            real_time_str = real_time.isoformat() if isinstance(real_time, datetime) else str(real_time)
            cursor.execute("SELECT * FROM SUMMARY_REPORT WHERE real_time=?",(real_time_str,))

            row=cursor.fetchone()
            return dict(row) if row else None 

    def get_latest_summarization(self) -> Optional[dict[str, Any]]:
        with sqlite3.connect(self.db_path) as conn:
             conn.row_factory = sqlite3.Row
             cursor = conn.cursor()

             cursor.execute("""
            SELECT * FROM SUMMARY_REPORT
            ORDER BY real_time DESC
            LIMIT 1
        """)
             row = cursor.fetchone()
             return dict(row) if row else None
       



#db=DatabaseManager(Database_URL)

Sending message

In [16]:


def send_whatsapp_message(to_number, message_body):
    try:
        account_sid = os.environ["TWILIO_ACCOUNT_SID"]
        auth_token = os.environ["TWILIO_AUTH_TOKEN"]
        client = Client(account_sid, auth_token)
        
        message = client.messages.create(
            from_="whatsapp:+14155238886",
            to=f"whatsapp:{to_number}",
            body=message_body
        )
        
        print(f"✅ Message sent successfully! SID: {message.sid}")
        return True
        
    except TwilioException as e:
        print(f"❌ Twilio error: {e}")
        return False
    except Exception as e:
        print(f"❌ General error: {e}")
        return False
                                                             
# Usage
success = send_whatsapp_message("+919410046073","Hello Saksham")


✅ Message sent successfully! SID: SM6f6acc16db95b669d1489f84725663b7


Defining_A_Node for threat analysis

In [17]:
class Workflow():
    def __init__(self,db:DatabaseManager):
        self.llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)
        self.db=db

        self.template1 = """
You are an AI Assistant who is specially designed to analyse the bas64 format of audio files which is received from the phone of a woman who might get harassed at {real_time}. You have to generate a threat level between 5 to 10 based on the following file. If this prompt is running which exclusively means that there is night time and user input already a dangerous place:

Audio converted base64 file: {audiob64}

Remember:
Provide high rating if you found some distress sentences like "don't touch me", "I will complain about you", "Please don't do this to me"... and many other distress sentences.

Return your response in the exact JSON format:
{{
    "summary": "Provide a proper summary for the event",
    "threat_level":5.0-10.0,
    "distress_words": ["list", "of", "distress", "words"]
}}
provide no markdown.
"""

        self.template2 = """
You are an AI Assistant who is specially designed to analyse the base64 format of audio files which is received from the phone of a woman who is not around the peak night time which is at. You have to generate a threat level between 4 to 10 based on the following transcript and since some relaxation would be there as this is not night time. Thus, threat level could be low but provide high level if you hear serious distress words that you think might lead something dangerous to the woman:

Audio converted base64 file: {audiob64}

Remember:
Provide high rating if you found some distress sentences like "don't touch me", "I will complain about you", "Please don't do this to me"... and many other distress sentences.

Return your response in the exact JSON format:
{{
    "summary": "Provide a proper summary for the event",
    "threat_level": 4.0-8.0,
    "distress_words": ["list", "of", "distress", "words"]
}}
Provide no markdown.
"""

    async def Analysis_Node(self, state: WorkflowState):
        night_start = dtime(0, 0)    # 00:00 (12:00 AM)
        night_end = dtime(4, 0)
        is_night=night_start<=state.real_time.time()<=night_end

        if (
            state.consent == 'No'
            and not is_night
            and state.gesture == 'No'
            and state.sudden_acc == 'No'
        ):
            return 'END'

        # Use the correct template based on night time
        if is_night:
            template = self.template1
        else:
            template = self.template2


        message = HumanMessage(
         content=[
        {"type": "text", "text":template},
        {"type": "audio", "source_type": "base64", "data": state.audio_b64, "mime_type": "audio/wav"}
    ]
)
        response = self.llm.invoke([message])            

        

        try:
            result = json.loads(response.content)
            obj = Summarization(
            location=state.location,
            time=state.real_time,            # Pydantic will parse ISO string to datetime
            summary=result["summary"],
            threat_level=float(result["threat_level"]),
            distress_words=result.get("distress_words",[]),
    )
            
            return {"analysis": obj,"threat_score":obj.threat_level} 
        except Exception as e:
            obj= Summarization(
                location=state.location,
                summary=f"Groq API error: {str(e)}",
                time=state.real_time,
                distress_words=[],
                threat_level=0.0
            )
            return {"analysis":obj,"threat_score":0.0}
        
    async def Just_Keep_logs(self,state:WorkflowState)->WorkflowState: 
            self.db.store_logs(
                state.real_time,
                state.location,
                state.TAudio,
                state.consent,
                state.gesture,
                state.sudden_acc,
                state.message            
          )
            return state

             
    async def Alarming_node(self,state:WorkflowState)->WorkflowState:
             #Storing the result and would send all the details to certain numbers    and store the logs and summary in database alongwith audio  
             self.db.store_logs(
                state.real_time,
                state.location,
                state.TAudio,
                state.consent,
                state.gesture,
                state.sudden_acc,
                state.message )
             obj=state.analysis
             if obj:
               self.db.store_summary(
                  real_time=obj.time,
                  threat_level=obj.threat_level,
                  summary=obj.summary,
                  location=obj.location,
                  #audio=state.audio_b64
                  distress_words=obj.distress_words
                    )
             import json
             body = json.dumps(self.db.get_summarization(state.real_time))
             send_whatsapp_message("+919410046073", body)
             return state
    
    async def Severe_Situation(self,state:WorkflowState)->WorkflowState:
              self.db.store_logs(
                real_time=state.real_time,
                location=state.location,
                TAudio=state.TAudio,
                consent=state.consent,
                gesture=state.gesture,
                sudden_acc=state.sudden_acc,
                message=state.message)
              
              obj=state.analysis
              self.db.store_summary(
                  real_time=obj.time,
                  threat_level=obj.threat_level,
                  summary=obj.summary,
                  location=obj.location,
                  audio=state.audio_b64,
                  distress_words=obj.distress_words
                    )
              import json
              body = json.dumps(self.db.get_summarization(state.real_time))
              send_whatsapp_message("+919410046073", body)
              print("A function that would send a call and save the audio")
              return state
    async  def check_condition(self,state:WorkflowState,config=None):

       if state.threat_score < 7:
         return "Just_Keep_logs"
       elif state.threat_score<8.5:
         return "Alarming_node"
       else:
        return "Severe_Situation"
    async def Flow(self):
        workflow = StateGraph(WorkflowState)
        # Adding the Nodes
        workflow.add_node("Analysis", self.Analysis_Node)
        workflow.add_node("Severe_Situation",self.Severe_Situation)
        workflow.add_node("Alarming_node",self.Alarming_node)
        workflow.add_node("Just_Keep_logs",self.Just_Keep_logs)


        workflow.set_entry_point("Analysis")
        workflow.add_conditional_edges("Analysis",self.check_condition)
        workflow.add_edge("Severe_Situation",END)
        workflow.add_edge("Alarming_node",END)
        workflow.add_edge("Just_Keep_logs",END)

        return workflow.compile()

Need to create API_EndPoints

In [18]:
Database_URL = "Log_of_Events.db"
db = DatabaseManager(Database_URL)

    # Create workflow instance
workflow_instance = Workflow(db=db)
graph_app = await workflow_instance.Flow()
    

In [19]:


app = FastAPI(title="Safety Alert API", version="1.0")



@app.get("/", summary="Health Check")
def health_check():
    return {"message": "Safety Alert API is running"}

@app.post(
    "/providing-inputs",
    summary="Submit alert data with audio upload",
    #response_model=WorkflowState,
    #response_model_exclude_none=True
)
async def providing_inputs(
    # Location fields
    latitude: float = Form(..., description="Latitude coordinate"),
    longitude: float = Form(..., description="Longitude coordinate"),
    # WorkflowState fields (real_time omitted to auto-generate)
    threat_score: float = Form(0.0, description="Initial threat score"),
    consent: Literal["Yes", "No"] = Form(..., description="User consent"),
    gesture: Literal["Yes", "No"] = Form(..., description="Gesture detected"),
    sudden_acc: Literal["Yes", "No"] = Form(..., description="Sudden acceleration"),
    message: str = Form(..., description="Alert message"),
    # Audio file upload
    audio_file: UploadFile = File(..., description="Audio file to convert to base64")
):
    

    # 1. Read & validate audio file
    allowed_ext = {"mp3", "wav", "m4a", "ogg", "flac", "aac"}
    ext = audio_file.filename.rsplit(".", 1)[-1].lower() if "." in audio_file.filename else ""
    if not (audio_file.content_type.startswith("audio/") or ext in allowed_ext):
        raise HTTPException(
            status_code=400,
            detail=f"Unsupported audio format. Allowed: {sorted(allowed_ext)}"
        )
    content = await audio_file.read()
    audio_b64 = base64.b64encode(content).decode()

    # 2. Auto-generate timestamp


    # 3. Build models
    #loc = Location(latitude=latitude, longitude=longitude)
    state = WorkflowState(
        location=Location(latitude=latitude, longitude=longitude),
        real_time=datetime.now(),
        TAudio=None,
        threat_score=threat_score,
        consent=consent,
        gesture=gesture,
        sudden_acc=sudden_acc,
        message=message,
        audio_b64=audio_b64
    )
    result = await graph_app.ainvoke(state)

    print(type(result))
    print(result['analysis'])
    print(result['analysis'].threat_level)
    return {"Status":"Ok"}


@app.get("/get-latest-summary", summary="Fetch latest summary report")
async def get_latest_summary():
    try:
        row=db.get_latest_summarization()
        return {"Report":row}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))


    

In [20]:
import uvicorn
import nest_asyncio

# This allows uvicorn to run in Jupyter
nest_asyncio.apply()

# Run the server
uvicorn.run(app, host="localhost", port=8300, reload=False)


INFO:     Started server process [24848]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://localhost:8300 (Press CTRL+C to quit)


INFO:     ::1:57863 - "GET / HTTP/1.1" 200 OK
INFO:     ::1:57864 - "GET / HTTP/1.1" 200 OK
INFO:     ::1:57864 - "GET /docs HTTP/1.1" 200 OK
INFO:     ::1:57864 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     ::1:57908 - "GET /get-latest-summary HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [24848]
